In [1]:
# 导入数据分割， 模型验证，cv参数搜索，以及lightgbm包
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import GridSearchCV
import lightgbm as lgbm

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
%%time
data = pd.read_parquet("C:\\Users\\Mengxiao.Wu\\Desktop\\train\\train_low_mem.parquet")
data.describe()

Wall time: 55.6 s


,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,f_6,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
count,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,...,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06
mean,6.654862e+02,1.885265e+03,-2.109159e-02,1.637057e-03,-1.257678e-02,8.514749e-03,-2.003703e-02,-5.416438e-03,-3.034008e-03,1.934330e-03,...,1.256709e-02,9.002053e-04,-6.509154e-03,2.851608e-03,-4.730820e-03,-2.773806e-02,-1.517383e-03,-7.057928e-03,-4.216896e-03,-4.319488e-03
std,3.560434e+02,1.083616e+03,9.176629e-01,1.079721e+00,1.030980e+00,1.030281e+00,9.602916e-01,9.895188e-01,1.104578e+00,1.067755e+00,...,1.084496e+00,1.114387e+00,1.070176e+00,1.101158e+00,1.144516e+00,9.592550e-01,1.140459e+00,1.108097e+00,1.051252e+00,1.008220e+00
min,0.000000e+00,0.000000e+00,-9.419646e+00,-1.765789e+01,-6.579473e+00,-8.644268e+00,-1.800427e+01,-4.000015e+00,-8.833704e+00,-6.384251e+00,...,-9.271487e+00,-6.359966e+00,-8.183732e+00,-2.399478e+01,-1.497270e+01,-9.348986e+00,-7.366648e+00,-7.579406e+00,-6.707284e+00,-1.028264e+01
25%,3.530000e+02,9.520000e+02,-5.004572e-01,-4.071441e-01,-6.813878e-01,-6.530838e-01,-4.496046e-01,-3.532303e-01,-7.070855e-01,-7.911198e-01,...,-6.037326e-01,-8.355157e-01,-6.981400e-01,-1.377131e-01,-9.397985e-01,-5.147926e-01,-9.573807e-01,-7.239380e-01,-8.165390e-01,-5.080143e-01
50%,7.040000e+02,1.882000e+03,-9.717009e-02,2.431158e-01,-4.867587e-02,4.350941e-02,-2.639937e-01,-1.908876e-01,-2.144796e-02,1.437945e-02,...,6.086323e-02,-2.023181e-01,-1.910102e-01,2.164071e-01,2.087202e-01,-2.907780e-01,6.955573e-03,-1.650222e-01,2.300689e-02,-2.824031e-01
75%,9.750000e+02,2.830000e+03,3.572908e-01,6.649507e-01,6.086557e-01,6.587324e-01,7.763371e-02,2.784694e-02,6.835684e-01,8.008306e-01,...,6.882384e-01,9.035954e-01,4.841421e-01,5.088849e-01,8.616266e-01,1.275946e-01,9.503851e-01,6.098197e-01,7.984827e-01,1.411301e-01
max,1.219000e+03,3.773000e+03,1.203861e+01,7.845261e+00,8.707207e+00,8.009340e+00,4.706333e+01,7.662866e+01,7.646200e+00,6.778142e+00,...,9.298274e+00,9.725060e+00,2.231185e+01,6.587691e+00,6.978151e+00,6.140367e+01,7.679950e+00,1.241804e+01,7.003982e+00,4.337021e+01


In [3]:
data['investment_id'] = data['investment_id'].astype('category')
feats = [col for col in data.columns if col not in ['time_id', 'row_id', 'target']]
X = data[feats]
y = data.target

In [8]:
%%time
param = {
      'n_estimators': range(50,1000,50),
    }
estimator = lgbm.LGBMRegressor(boosting_type='gbdt',
                               objective='regression',
                               metrics='rmse',
                               learning_rate=0.1)
gsearch = GridSearchCV(estimator, param_grid=param, scoring='neg_root_mean_squared_error', cv=5, n_jobs=8, verbose=1)
gsearch.fit(X, y)
print("*********************")
print(gsearch.best_params_)
print(gsearch.best_score_)

Fitting 5 folds for each of 19 candidates, totalling 95 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 36.9min
[Parallel(n_jobs=8)]: Done  95 out of  95 | elapsed: 190.5min finished


*********************
{'n_estimators': 100}
-0.9145781284100764
Wall time: 3h 11min 51s


In [4]:
%%time
param = {
     'max_depth': range(7,25,2)    
    }
estimator = lgbm.LGBMRegressor(boosting_type='gbdt',
                               objective='regression',
                               metrics='rmse',
                               learning_rate=0.1,
                               n_estimators=100,
                              )
gsearch = GridSearchCV(estimator, param_grid=param, scoring='neg_mean_squared_error', cv=5, n_jobs=8, verbose=1)
gsearch.fit(X, y)
print("*********************")
print(gsearch.best_params_)
print(gsearch.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  45 out of  45 | elapsed: 31.1min finished


*********************
{'max_depth': 7}
-0.8363557172633274
Wall time: 32min 28s


In [5]:
%%time
param = {
     'num_leaves': range(5, 150, 5)  
    }
estimator = lgbm.LGBMRegressor(boosting_type='gbdt',
                               objective='regression',
                               metrics='rmse',
                               learning_rate=0.1,
                               n_estimators=100,
                               max_depth=12,
                              )
gsearch = GridSearchCV(estimator, param_grid=param, scoring='neg_root_mean_squared_error', cv=5, n_jobs=8, verbose=1)
gsearch.fit(X, y)
print("*********************")
print(gsearch.best_params_)
print(gsearch.best_score_)

Fitting 5 folds for each of 29 candidates, totalling 145 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 22.3min
[Parallel(n_jobs=8)]: Done 145 out of 145 | elapsed: 106.8min finished


*********************
{'num_leaves': 10}
-0.9141089709654124
Wall time: 1h 47min 53s


In [4]:
%%time
param = {
      'feature_fraction': [0.6, 0.7, 0.8, 1],
    }

estimator = lgbm.LGBMRegressor(boosting_type='gbdt',
                               objective='regression',
                               metrics='rmse',
                               learning_rate=0.1,
                               n_estimators=200,
                               max_depth=5,
                               num_leaves=5,
                              )
gsearch = GridSearchCV(estimator, param_grid=param, scoring='neg_root_mean_squared_error', cv=5, n_jobs=8, verbose=1)
gsearch.fit(X, y)
print("*********************")
print(gsearch.best_params_)
print(gsearch.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed: 17.2min finished


[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
*********************
{'feature_fraction': 0.6}
-0.9135283761415727
Wall time: 18min 21s


In [5]:
%%time
param = {
     'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0],
     'bagging_freq':    range(0, 6),
    }
estimator = lgbm.LGBMRegressor(boosting_type='gbdt',
                               objective='regression',
                               metrics='rmse',
                               learning_rate=0.1,
                               n_estimators=200,
                               max_depth=5,
                               num_leaves=5,
                               feature_fraction=0.6,
                              )
gsearch = GridSearchCV(estimator, param_grid=param, scoring='neg_root_mean_squared_error', cv=5, n_jobs=8, verbose=1)
gsearch.fit(X, y)
print("*********************")
print(gsearch.best_params_)
print(gsearch.best_score_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 18.3min
[Parallel(n_jobs=8)]: Done 150 out of 150 | elapsed: 76.3min finished


[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
*********************
{'bagging_fraction': 0.6, 'bagging_freq': 0}
-0.9135283761415727
Wall time: 1h 17min 36s


In [5]:
%%time
params_test2 = {'max_bin': range(150, 256, 10), 'min_data_in_leaf': range(1, 202, 10)}
estimator = lgbm.LGBMRegressor(boosting_type='gbdt',
                               objective='regression',
                               metrics='rmse',
                               learning_rate=0.1,
                               n_estimators=200,
                               feature_fraction=0.6,
                              )
gsearch = GridSearchCV(estimator, param_grid=param, scoring='neg_root_mean_squared_error', cv=5, n_jobs=8, verbose=1)
gsearch.fit(X, y)
print("*********************")
print(gsearch.best_params_)
print(gsearch.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  45 out of  45 | elapsed: 36.6min finished


*********************
{'max_depth': 7}
-0.9132743973572447
Wall time: 38min 13s
